<a href="https://colab.research.google.com/github/5amessi/Binary-Classification/blob/master/Binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import math
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
from keras.models import Sequential
from keras import regularizers
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import RMSprop
from keras.initializers import glorot_uniform, glorot_normal, RandomUniform
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import timedelta
from sklearn.externals import joblib 
import datetime

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
def Clean(data,valid = False):
  data2 = data.index
  data = data.values
  new_data = []
  for i in range(len(data)):
    if(str(data[i][0]) == str(1e400*0)):
      if(str(data2[i][-1]) != str(1e400*0)):
        new_line3 = list(map(str,data2[i][2].split(';')))   
        if len(new_line3) == 17:
          new_line1 = list(map(str,data2[i][0].split(';')))
          new_line2 = list(map(str,data2[i][1].split(';')))
          new_data.append(new_line1 + new_line2 + new_line3[:6]+['']+new_line3[6:]) 
          pass
        pass
      pass
    else:
      new_line1 = list(map(str,data2[i][0].split(';')))
      new_line2 = list(map(str,data2[i][1].split(';')))
      new_line3 = list(map(str,data2[i][2].split(';')))      
      new_line4 = list(map(str,data[i][0].split(';')))
      
      new_data.append(new_line1 + new_line2 + new_line3 + new_line4)
#       print(new_line1 + new_line2 + new_line3 + new_line4)
  data = pd.DataFrame(new_data,columns=['v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','target'])
  data.to_csv("data.csv",index=False)
  data = pd.read_csv("/content/data.csv")
  return data


In [117]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/training.csv")
print(len(data))
data = Clean(data)
print(len(data))
data.head()


3700
2685


,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,target
0,a,17,92,5,0.00004,"""u""","""g""","""c""","""v""",1,75.0,"""f""","""t""",1,"""t""","""g""",80.0,5,800000.0,"""t""",0,"""no."""
1,b,16,92,3,0.00035,"""y""","""p""","""k""","""v""",0,29.0,"""f""","""f""",0,"""f""","""s""",200.0,0,2000000.0,NaN,0,"""no."""
2,b,31,25,0,1125.00000,"""u""","""g""","""ff""","""ff""",0,NaN,"""f""","""t""",1,"""f""","""g""",96.0,19,960000.0,"""t""",0,"""no."""
3,a,48,17,0,1335.00000,"""u""","""g""","""i""","""o""",0,335.0,"""f""","""f""",0,"""f""","""g""",0.0,120,0.0,NaN,0,"""no."""
4,b,32,33,0,35.00000,"""u""","""g""","""k""","""v""",0,5.0,"""f""","""f""",0,"""t""","""g""",232.0,0,2320000.0,"""f""",0,"""no."""


In [119]:
for i in data:
  if (len(data[i].unique()) < 50):
    print(i,data[i].unique())

v0 ['a' 'b' nan]
v2 [92 25 17 33 83 75 58 67 42  5  8]
v3 [5 3 0 8 7 2 6 4 1 9]
v5 ['"u"' '"y"' '"l"']
v6 ['"g"' '"p"' '"gg"']
v7 ['"c"' '"k"' '"ff"' '"i"' '"q"' '"W"' '"d"' '"aa"' '"cc"' '"r"' '"m"'
 '"j"' '"e"' nan '"x"']
v8 ['"v"' '"ff"' '"o"' '"h"' '"bb"' '"z"' nan '"dd"' '"n"' '"j"']
v9 [ 1  0  2  5  4  6  3  8 15 14  7  9 12 13 10 16 17 28 20 11 18]
v10 [ 75.  29.  nan 335.   5.  25.  85. 165.   4. 585. 375. 125. 415. 665.
  21.  58.  71.  46. 625.  54. 835. 875.  96.  17.]
v11 ['"f"' '"t"']
v12 ['"t"' '"f"']
v13 [ 1  0  2  3  5 15 20 11 12  9  4 14  7  6 16  8 10 13 67 40 17]
v14 ['"t"' '"f"']
v15 ['"g"' '"s"' '"p"']
v19 ['"t"' nan '"f"']
v20 [0 1]
target ['"no."' '"yes."']


In [0]:
def Prepare(data):
  data['v0'] = data['v0'].fillna(50)
  data['v7'] = data['v7'].fillna(50)
  data['v8'] = data['v8'].fillna(50)
  data['v19'] = data['v19'].fillna(50)
  
  for i in range(len(data['v0'].unique())):
    data['v0'] = data['v0'].replace(data['v0'].unique()[i],i)
  
  for i in range(len(data['v5'].unique())):
    data['v5'] = data['v5'].replace(data['v5'].unique()[i],i)

  for i in range(len(data['v6'].unique())):
    data['v6'] = data['v6'].replace(data['v6'].unique()[i],i)

  for i in range(len(data['v7'].unique())):
    data['v7'] = data['v7'].replace(data['v7'].unique()[i],i)

  for i in range(len(data['v8'].unique())):
    data['v8'] = data['v8'].replace(data['v8'].unique()[i],i)
    
  for i in range(len(data['v11'].unique())):
    data['v11'] = data['v11'].replace(data['v11'].unique()[i],i)
    
  for i in range(len(data['v12'].unique())):
    data['v12'] = data['v12'].replace(data['v12'].unique()[i],i)
    
  for i in range(len(data['v14'].unique())):
    data['v14'] = data['v14'].replace(data['v14'].unique()[i],i)
    
  for i in range(len(data['v15'].unique())):
    data['v15'] = data['v15'].replace(data['v15'].unique()[i],i)
    
  for i in range(len(data['v19'].unique())):
    data['v19'] = data['v19'].replace(data['v19'].unique()[i],i)
    
  for i in range(len(data['target'].unique())):
    data['target'] = data['target'].replace(data['target'].unique()[i],i)
  data = data.fillna(data.mean())
  return data

In [121]:
data = Prepare(data)
data.head()

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,target
0,0,17,92,5,0.00004,0,0,0,0,1,75.000000,0,0,1,0,0,80.0,5,800000.0,0,0,0
1,1,16,92,3,0.00035,1,1,1,0,0,29.000000,0,1,0,1,1,200.0,0,2000000.0,1,0,0
2,1,31,25,0,1125.00000,0,0,2,1,0,160.949726,0,0,1,1,0,96.0,19,960000.0,0,0,0
3,0,48,17,0,1335.00000,0,0,3,2,0,335.000000,0,1,0,1,0,0.0,120,0.0,1,0,0
4,1,32,33,0,35.00000,0,0,1,0,0,5.000000,0,1,0,0,0,232.0,0,2320000.0,2,0,0


In [122]:
y = data['target']
x = data.drop(['target'],axis=1)
x.shape,y.shape

((2685, 21), (2685,))

In [123]:
scaler = StandardScaler()
x = scaler.fit_transform(x)
x.shape

(2685, 21)

In [0]:
activ_func = 'tanh'  
loss = 'binary_crossentropy'                  
optimizer="adam"              
dropout = 0.05                 

In [125]:
model = Sequential()
model.add(Dense(256,input_dim=np.shape(x)[1],activation=activ_func,kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(128,activation=activ_func))
model.add(Dense(units=1,activation = 'sigmoid'))
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               5632      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [129]:
model.fit(np.array(x), np.array(y), epochs=1, batch_size=128, verbose=1)

Epoch 1/1
2685/2685 [==============================] - 0s 70us/step - loss: 0.0436 - acc: 1.0000


In [127]:
valid = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/binary_classification/validation.csv")
valid = Clean(valid)
valid = Prepare(valid)
print(valid.head())
valid_y = valid['target']
valid_x = valid.drop(['target'],axis=1)
valid_x.shape,valid_y.shape

   v0  v1  v2  v3           v4  v5  ...  v16  v17        v18  v19  v20  target
0   0  32  33   0     75.00000   0  ...  420    0  4200000.0    0    1       0
1   0  23  58   0    179.00000   0  ...  136    1  1360000.0    0    0       0
2   0  36  42   7      0.00005   1  ...  240    3  2400000.0    0    1       0
3   0  18  42   0  10415.00000   1  ...  120  375  1200000.0    0    0       0
4   0  24   5   0  13335.00000   1  ...  120  475  1200000.0    1    1       0

[5 rows x 22 columns]


((155, 21), (155,))

In [130]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = model.predict_classes(valid_x)
result = precision_recall_fscore_support(valid_y, y_pred,average='binary')
print('precision=',result[0],'recall=',result[1], 'f1score=',result[2])

precision= 0.45806451612903226 recall= 1.0 f1score= 0.6283185840707964
